### 载入所需的库
我们依旧会用gensim去做word2vec的处理，会用sklearn当中的SVM进行建模

In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2015.12.09

@author: Hanxiaoyang
"""
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import jieba
from sklearn.externals import joblib
from sklearn.svm import SVC
import sys  
# reload(sys)  
# sys.setdefaultencoding('utf8')

C:\MyPrograms\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 载入数据，做预处理(分词)，切分训练集与测试集

In [2]:
def load_file_and_preprocessing():
    neg=pd.read_excel('data/neg.xls',header=None,index=None)
    pos=pd.read_excel('data/pos.xls',header=None,index=None)

    cw = lambda x: list(jieba.cut(x))
    pos['words'] = pos[0].apply(cw)
    neg['words'] = neg[0].apply(cw)

    #print pos['words']
    #use 1 for positive sentiment, 0 for negative
    y = np.concatenate((np.ones(len(pos)), np.zeros(len(neg))))

    x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos['words'], neg['words'])), y, test_size=0.2)
    
    np.save('svm_data/y_train.npy',y_train)
    np.save('svm_data/y_test.npy',y_test)
    return x_train,x_test

### 对每个句子的所有词向量取均值，来生成一个句子的vector

In [3]:
def build_sentence_vector(text, size, imdb_w2v):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += imdb_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

### 计算词向量

In [32]:
#计算词向量
def get_train_vecs(x_train,x_test):
    n_dim = 300
    #初始化模型和词表
    imdb_w2v = Word2Vec(size=n_dim, min_count=10)
    imdb_w2v.build_vocab(x_train)
    
    #在评论训练集上建模(可能会花费几分钟)
    imdb_w2v.train(x_train)
    
    train_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_train])
    #train_vecs = scale(train_vecs)
    
    np.save('svm_data/train_vecs.npy',train_vecs)
    print(train_vecs.shape)
    
    #在测试集上训练
    imdb_w2v.train(x_test)
    imdb_w2v.save('svm_data/w2v_model/w2v_model.pkl')
    #Build test tweet vectors then scale
    test_vecs = np.concatenate([build_sentence_vector(z, n_dim,imdb_w2v) for z in x_test])
    #test_vecs = scale(test_vecs)
    np.save('svm_data/test_vecs.npy',test_vecs)
    print(test_vecs.shape)

In [21]:
def get_data():
    train_vecs=np.load('svm_data/train_vecs.npy')
    y_train=np.load('svm_data/y_train.npy')
    test_vecs=np.load('svm_data/test_vecs.npy')
    y_test=np.load('svm_data/y_test.npy') 
    return train_vecs,y_train,test_vecs,y_test

### 训练svm模型

In [22]:
def svm_train(train_vecs,y_train,test_vecs,y_test):
    clf=SVC(kernel='rbf',verbose=True) #SVM分类器  rbf=高斯核
    clf.fit(train_vecs,y_train)
    joblib.dump(clf, 'svm_data/svm_model/model.pkl')
    print(clf.score(test_vecs,y_test))

### 构建待预测句子的向量

In [23]:
def get_predict_vecs(words):
    n_dim = 300
    imdb_w2v = Word2Vec.load('svm_data/w2v_model/w2v_model.pkl')
    #imdb_w2v.train(words)
    train_vecs = build_sentence_vector(words, n_dim,imdb_w2v)
    #print train_vecs.shape
    return train_vecs

### 对单个句子进行情感判断 

In [24]:
def svm_predict(string):
    words=jieba.lcut(string)
    words_vecs=get_predict_vecs(words)
    clf=joblib.load('svm_data/svm_model/model.pkl')
     
    result=clf.predict(words_vecs)
    
    if int(result[0])==1:
        print(string,' positive')
    else:
        print(string,' negative')

In [26]:
##对输入句子情感进行判断
string='电池充完了电连手机都打不开.简直烂的要命.真是金玉其外,败絮其中!连5号电池都不如'
string='牛逼的手机，从3米高的地方摔下去都没坏，质量非常好'    
# svm_predict(string)

In [27]:
# step-1
# 载入数据，做预处理(分词)，切分训练集与测试集
x_train,x_test = load_file_and_preprocessing()

In [28]:
print("x_train.shape = ", x_train.shape)
print("x_train[0] = ", x_train[0])
print("x_test.shape = ", x_test.shape)
print("x_test[0] = ", x_test[0])

x_train.shape =  (16884,)
x_train[0] =  ['虽然', '书中案', '列', '都', '比较', '特殊', '，', '或者', '比较', '夸张', '，', '可是', '，', '我', '个人', '还是', '觉得', '这是', '一本', '很', '有用', '的', '书', '。', '对', '本书', '的', '看法', '应该', '是', '选择', '自己', '有用', '的', '信息', '，', '没有', '必要', '去', '争议', '一些', '无聊', '的', '细节', '或者', '案列', '的', '真实性', '。', '每个', '人', '的', '生活习惯', '饮食', '方式', '都', '不', '一样', '，', '不', '可能', '书中', '所以', '方案', '都', '能', '为', '你', '所用', '，', '或者', '，', '都', '能', '为', '你', '所用', '，', '这', '本书', '不是', '天书', '也', '该', '天价', '了', '，', '所以', '，', '平常心', '，', '这', '本书', '的', '内容', '对', '的', '起', '这个', '价格', '。', '家中', '备', '一本', '，', '有用', '。']
x_test.shape =  (4221,)
x_test[0] =  ['一个', '同事', '向', '我', '推荐', '这', '本书', '，', '于是', '我', '满怀希望', '地', '跟', '当当', '订货', '，', '结果', '苦', '等', '了', '两周', '之后', '，', '寄来', '的', '东西', '根本', '不是', '这', '本书', '，', '他们', '又', '一次', '把', '货', '发错', '了', '，', '我', '退货', '之后', '至今', '仍', '未', '收到', '那本书', '，', '相当', '的', '郁闷', '啊', '！']


In [33]:
# step-2
# 计算词向量 (有问题没解决！)
# get_train_vecs(x_train,x_test)